In [ ]:
%%R 
srat_combined = readRDS('integrated_data.rds') # The output of SCTransform
HVG = VariableFeatures(srat_combined)

In [ ]:
%R mat <- srat_combined@assays$integrated@scale.data
# The SCT residuals

array([[-2.6937571 , -2.45442555, -2.43574886, ..., 10.        ,
        -0.22105828,  5.97981812],
       [-2.28048602, -3.18019692, -2.68573542, ...,  7.54522133,
        10.        , 10.        ],
       [-0.94005987,  1.67083964, -0.40994753, ..., -0.71351776,
         7.70982643, -0.64411865],
       ...,
       [ 0.1888066 , -0.36333327, -0.23790177, ..., -0.28515844,
        -0.55464638, -0.42498454],
       [-0.14404412, -0.23095836, -0.3972117 , ..., -0.26328313,
        -0.22352942, -0.46534847],
       [-0.11213089,  0.03447263,  0.03982213, ...,  0.01214038,
        -0.2127177 , -0.26438226]])

In [ ]:
%%R -o metadata
metadata = srat_combined@meta.data
#get the metadata

In [ ]:
%%R
srat_combined <- RunPCA(srat_combined)
pca <- srat_combined[["pca"]]

# Get the total variance:
total_variance <- sum(matrixStats::rowVars(mat))

eigValues = (pca@stdev)^2  ## EigenValues
varExplained = eigValues / total_variance

PCs = Loadings(srat_combined, reduction = "pca")

In [ ]:
%%R -o logcounts -o counts
counts = srat_combined[["SCT"]]@counts #SCT corrected counts
logcounts = srat_combined[["SCT"]]@data # log1p SCT corrected counts

In [ ]:
%%R -o features -o HVG -o varExplained -o PCs -o mat -o obs_names -o PC_embeddings
features = rownames(srat_combined[["SCT"]])
obs_names = colnames(srat_combined[["SCT"]])
HVG = rownames(srat_combined@assays$integrated)
PC_embeddings = srat_combined@reductions$pca@cell.embeddings

In [ ]:
#Remeber to do the transpose when back from R to python!
import anndata
adata = anndata.AnnData(X = logcounts.T)
adata.var_names = features
adata.obs_names = obs_names
adata.obs = metadata
adata.uns['scaled'] = mat.T 
adata.layers['counts'] = counts.T
adata.uns['residuals_genes'] = HVG
adata.uns['residuals_genes'] = list(HVG)
adata.layers['logcounts'] = logcounts.T
adata.obsm['X_pca'] = PC_embeddings

In [ ]:
adata.obs['n_counts'] = adata.layers['counts'].sum(1)
adata.obs['n_genes'] = (adata.layers['counts'] > 0).sum(1)
adata.var['highly_variable'] = adata.var_names.isin(HVG)
adata.uns['pca'] = dict({'variance_ratio': varExplained})